# Subgroups and Lagrange's Theorem

**Module 01e** | Modular Arithmetic and Groups

*The most important theorem in elementary group theory: a subgroup's size MUST divide the group's size.*

> **Question:** In $\mathbb{Z}/12\mathbb{Z}$, the element 4 generates $\{0, 4, 8\}$, a group of size 3 inside a group of size 12. Could it have generated a group of size 5? Of size 7? What sizes are *possible*?

## Objectives

By the end of this notebook you will be able to:

1. **Generate** a subgroup from a single element.
2. **State and verify** Lagrange's theorem.
3. **Compute cosets** and verify they partition the group.
4. **Apply** Lagrange's theorem to constrain element orders.

## From Generators to Subgroups

In [01d](01d-cyclic-groups-generators.ipynb) we saw that non-generators produce smaller cycles, elements like 2 in $(\mathbb{Z}/7\mathbb{Z}^*)$ generate only $\{1, 2, 4\}$, not the whole group. These smaller cycles are actually groups in their own right: **subgroups**.

Lagrange's theorem says their sizes are brutally constrained: a subgroup's size MUST divide the group's size. This is arguably the most important theorem in elementary group theory.

## Subgroups by Generation

The subgroup generated by element $a$ in $(\mathbb{Z}/n\mathbb{Z}, +)$ is $\langle a \rangle = \{a, 2a, 3a, \ldots\}$, keep adding $a$ until you return to 0.

In [ ]:
# Generate the subgroup <a> for every element a in Z/12Z
n = 12

def generate_subgroup_additive(a, n):
    """Compute the subgroup <a> in (Z/nZ, +)."""
    subgroup = set()
    x = 0
    while True:
        x = (x + a) % n
        subgroup.add(x)
        if x == 0:
            break
    return sorted(subgroup)

print(f'Subgroups of (Z/{n}Z, +) generated by each element:')
print('a  <a>  size')for a in range(n):
    sg = generate_subgroup_additive(a, n)
    print(f'{a}  {str(sg)}  {len(sg)}')

> **Checkpoint:** Before computing, predict $|\langle 3 \rangle|$ in $\mathbb{Z}/12\mathbb{Z}$. How many steps until $3k \equiv 0 \pmod{12}$?
>
> *Hint:* $|\langle a \rangle| = n / \gcd(a, n)$. So $|\langle 3 \rangle| = 12 / \gcd(3, 12) = 12/3 = 4$.

## Lagrange's Theorem: The Constraint

**Theorem (Lagrange):** If $H$ is a subgroup of $G$, then $|H|$ divides $|G|$.

Let's verify this for every subgroup we just found.

In [ ]:
# Verify Lagrange's theorem: every subgroup size divides |G| = 12
n = 12
divs_of_n = divisors(n)
print(f'Divisors of {n}: {divs_of_n}\n')

# Collect distinct subgroups (as frozensets) and their sizes
subgroup_sizes = set()
distinct_subgroups = dict()  # size -> subgroup elements

for a in range(n):
    sg = generate_subgroup_additive(a, n)
    sg_frozen = frozenset(sg)
    sz = len(sg)
    subgroup_sizes.add(sz)
    if sz not in distinct_subgroups:
        distinct_subgroups[sz] = sg

print('Subgroup sizes that appear:', sorted(subgroup_sizes))
print(f'Divisors of {n}:             {divs_of_n}')
print()

# Verify each size divides 12
for sz in sorted(subgroup_sizes):
    assert n % sz == 0, f'{sz} does not divide {n}!'
    print(f'  |H| = {sz}  divides {n}?  {n}/{sz} = {n // sz}  ✓')

print(f'\nEvery divisor of {n} corresponds to a subgroup (since Z/{n}Z is cyclic).')

## Cosets: Why Lagrange Works

The proof of Lagrange uses **cosets**. Given a subgroup $H$ and an element $a$, the coset $a + H = \{a + h : h \in H\}$ is a "shifted copy" of $H$.

In [ ]:
# Compute all cosets of H = <4> = {0, 4, 8} in Z/12Z
n = 12
H = generate_subgroup_additive(4, n)
print(f'Subgroup H = <4> = {H}, |H| = {len(H)}\n')

# Build distinct cosets
covered = set()
cosets = []

for a in range(n):
    if a in covered:
        continue
    coset = sorted([(a + h) % n for h in H])
    cosets.append((a, coset))
    covered.update(coset)

print(f'Cosets of H in Z/{n}Z:')
for rep, coset in cosets:
    print(f'  {rep} + H = {coset}')

# Verify the cosets partition Z/12Z
all_elements = set()
for _, coset in cosets:
    coset_set = set(coset)
    assert len(all_elements & coset_set) == 0, 'Cosets overlap!'
    all_elements |= coset_set

assert all_elements == set(range(n)), 'Cosets do not cover all elements!'
print(f'\nNumber of cosets: {len(cosets)}')
print(f'Each coset has size: {len(H)}')
print(f'Partition check: {len(cosets)} cosets x {len(H)} elements = {len(cosets) * len(H)} = |G|  \u2713')

Four cosets, each of size 3, perfectly partition the 12 elements:
$$|G| = |H| \times (\text{number of cosets}) = 3 \times 4 = 12$$

The number of cosets $[G:H]$ is called the **index** of $H$ in $G$. Lagrange's theorem is just the statement that this partition always works: $|G| = |H| \cdot [G:H]$.

## Consequences for Element Orders

Since $\langle a \rangle$ is a subgroup, $\text{ord}(a) = |\langle a \rangle|$ must divide $|G|$.

In [ ]:
# In Z/12Z (additive), show that element orders can only be divisors of 12
n = 12
divs = divisors(n)
print(f'Divisors of {n}: {divs}')
print(f'So element orders can only be: {divs}\n')

print(f'a  ord(a)  divides {n}?')
for a in range(n):
    if a == 0:
        order = 1  # identity has order 1
    else:
        order = n // gcd(a, n)
    divides = 'yes' if n % order == 0 else 'NO!'
    print(f'{a}  {order}  {divides}')

# Collect all orders that actually appear
actual_orders = sorted(set(
    1 if a == 0 else n // gcd(a, n) for a in range(n)
))
print(f'\nOrders that appear: {actual_orders}')
print(f'Divisors of {n}:     {divs}')
print(f'Match: {actual_orders == divs}  (every divisor appears because Z/{n}Z is cyclic)')

> **Checkpoint:** What are the possible element orders in $(\mathbb{Z}/15\mathbb{Z}^*, \times)$? First find $|\mathbb{Z}/15\mathbb{Z}^*| = \varphi(15)$, then list its divisors.

In [ ]:
# Multiplicative group (Z/15Z)*
n = 15
phi_n = euler_phi(n)
print(f'phi({n}) = {phi_n}')
print(f'Divisors of {phi_n}: {divisors(phi_n)}')
print(f'So possible element orders: {divisors(phi_n)}\n')

# Compute actual orders of all units mod 15
units = [a for a in range(1, n) if gcd(a, n) == 1]
print(f'Units mod {n}: {units}\n')

print('a  ord(a)')print('-' * 15)
actual_orders = set()
for a in units:
    order = Mod(a, n).multiplicative_order()
    actual_orders.add(order)
    print(f'{a}  {order}')

print(f'\nOrders that appear: {sorted(actual_orders)}')
print(f'Divisors of {phi_n}:      {divisors(phi_n)}')

missing = set(divisors(phi_n)) - actual_orders
if missing:
    print(f'Missing divisors: {sorted(missing)}, (Z/{n}Z)* is not cyclic!')
else:
    print('All divisors appear.')

> **Common mistake:** "Lagrange says every divisor of $|G|$ appears as a subgroup size." **Not quite!** Lagrange says subgroup sizes *must be divisors*, but for non-cyclic groups, some divisors may have no corresponding subgroup. However, for cyclic groups like $\mathbb{Z}/n\mathbb{Z}$, every divisor $d$ of $n$ does give a subgroup of size $d$: namely $\langle n/d \rangle$.

## Exercises

### Exercise 1 (Worked)
Find ALL subgroups of $(\mathbb{Z}/12\mathbb{Z}, +)$. For each, give a generator and list its elements.

In [ ]:
# Exercise 1 (Worked), All subgroups of Z/12Z
n = 12

# Collect all distinct subgroups
seen = set()  # frozensets of subgroups we've already found
subgroups = []

for a in range(n):
    sg = generate_subgroup_additive(a, n)
    sg_key = frozenset(sg)
    if sg_key not in seen:
        seen.add(sg_key)
        subgroups.append((a, sg))

# Sort by subgroup size for clean display
subgroups.sort(key=lambda x: len(x[1]))

print(f'All distinct subgroups of (Z/{n}Z, +):\n')
print('Generator  Subgroup  Size  12/size')for gen, sg in subgroups:
    print(f'<{gen}>  {str(sg)}  {len(sg)}  {n // len(sg)}')

print(f'\nTotal: {len(subgroups)} subgroups (one for each divisor of {n})')

### Exercise 2 (Guided)
Compute the cosets of $H = \langle 5 \rangle$ in $(\mathbb{Z}/15\mathbb{Z}, +)$. Verify they partition $\mathbb{Z}/15\mathbb{Z}$. How many cosets are there?

In [ ]:
# Exercise 2: Cosets of <5> in Z/15Z
n = 15
# Step 1: compute the subgroup H = <5>
H = set()
x = 5
while True:
    H.add(x % n)
    x = (x + 5) % n
    if x % n in H:
        break
H.add(0)
H = sorted(H)
print(f'H = <5> = {H}, |H| = {len(H)}')

# Step 2: TODO, compute cosets a + H for a = 0, 1, 2, ...
# until you've covered all of Z/15Z

# Step 3: TODO, verify the cosets partition {0, 1, ..., 14}

### Exercise 3 (Independent)
In $(\mathbb{Z}/31\mathbb{Z}^*, \times)$, the group has order 30. List all possible subgroup sizes (= divisors of 30). For each size $d$, find an element whose order is $d$. Is there always such an element?

In [ ]:
# Exercise 3: Your code here


## Summary

- $\langle a \rangle = \{a, 2a, \ldots\}$ is always a subgroup; its size is $|G|/\gcd(a, |G|)$ for additive groups
- **Lagrange's theorem**: $|H|$ divides $|G|$, so subgroup sizes are constrained to divisors
- **Cosets** partition $G$ into equal-sized pieces: $|G| = |H| \times [G:H]$
- Element orders are also constrained: $\text{ord}(a)$ must divide $|G|$

> **Crypto teaser:** Lagrange's theorem is why **safe primes** matter. If $p = 2q + 1$ with $q$ prime, then $(\mathbb{Z}/p\mathbb{Z}^*)$ has only subgroups of order $1, 2, q, 2q$. An attacker can't exploit small subgroups, there aren't any (except the trivial ones).

**Next:** [Visualizing Group Structure](01f-group-visualization.ipynb), see everything we've learned as pictures.